In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Library is need

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

# Read data

In [ ]:
sample_submission = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
sample = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")

# Exploratory , analysis data

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.corr()['SalePrice'].sort_values()

In [ ]:
sns.scatterplot(data=train, x='OverallQual', y='SalePrice')
plt.axhline(y=200000,color='r')

In [ ]:
train[(train['OverallQual']>8) &(train['SalePrice']<200000)][['SalePrice', 'OverallQual']]

In [ ]:
sns.scatterplot(x='GrLivArea', y='SalePrice', data=train)
plt.axhline(y=200000, color='r')
plt.axvline(x=4000, color='r')

In [ ]:
train[(train['GrLivArea']>4000) & (train['SalePrice']<400000)][['SalePrice', 'GrLivArea']]

In [ ]:
index_drop=train[(train['GrLivArea']>4000) & (train['SalePrice']<400000)].index
train=train.drop(index_drop, axis=0)

In [ ]:
sns.scatterplot(x='GrLivArea', y='SalePrice', data=train)
plt.axhline(y=200000, color='r')
plt.axvline(x=4000, color='r')

In [ ]:
sns.scatterplot(x='OverallQual', y='SalePrice', data=train)
plt.axhline(y=200000,color='r')

In [ ]:
sns.boxplot(x='OverallQual', y='SalePrice', data=train)

In [ ]:
#open the documentation about the data set. (this file is in a txt format)

with open('../input/house-prices-advanced-regression-techniques/data_description.txt', 'r') as f:
    print(f.read())

In [ ]:
train= train.drop('Id', axis=1)

# Dropping null data

In [ ]:
train.isnull()

In [ ]:
#How many missing data is there in each features?
train.isnull().sum()

In [ ]:
#The percent of missing data in any feature:
100*(train.isnull().sum()/len(train))

In [ ]:
#Make a Function to calculate the percent of missing data in each columns (feature) and then sort it
def missing_percent(train):
    nan_percent= 100*(train.isnull().sum()/len(train))
    nan_percent= nan_percent[nan_percent>0].sort_values()
    return nan_percent

In [ ]:
nan_percent= missing_percent(train)

In [ ]:
nan_percent

In [ ]:
#plot the feature with missing indicating the percent of missing data
plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
#every Feature with missing data must be checked!
#We choose a threshold of 1%. It means, if there is less than 1% of a feature are missing,
#then we will consider just dropping that rows

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

#Set 1% threshold:
plt.ylim(0,1)

In [ ]:
nan_percent[nan_percent<1]

In [ ]:
nan_percent[nan_percent<1].index

In [ ]:
100/len(train)
#It shows that, Feature with just one missing rows has this percent value of missing data

In [ ]:
train[train['Electrical'].isnull()]

In [ ]:
train[train['GarageArea'].isnull()]

In [ ]:
train= train.dropna(axis=0, subset=['Electrical', 'GarageArea','MasVnrArea'])

In [ ]:
nan_percent= missing_percent(train)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)
plt.ylim(0,1)

In [ ]:
train[train['TotalBsmtSF'].isnull()]

In [ ]:
train[train['BsmtHalfBath'].isnull()]

In [ ]:
train[train['BsmtFullBath'].isnull()]

In [ ]:
#After checking the data documentation,
#it shows that missing value (two rows) in Basement Features are because of there is no basement in these rows
#Decision: Filling in data based on column: numerical basement & string descriptive:

#Numerical Columns fill with 0:
bsmt_num_cols= ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF' ,'BsmtFullBath', 'BsmtHalfBath']
train[bsmt_num_cols]=train[bsmt_num_cols].fillna(0)

#String Columns fill with None:
bsmt_str_cols= ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
train[bsmt_str_cols]= train[bsmt_str_cols].fillna('None')

In [ ]:
nan_percent= missing_percent(train)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)
plt.ylim(0,1)

In [ ]:
train["MasVnrType"]= train["MasVnrType"].fillna("None")
train["MasVnrArea"]= train["MasVnrArea"].fillna(0)

In [ ]:
nan_percent= missing_percent(train)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
train[['GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond']]

In [ ]:
#Filling the missing Value:
Gar_str_cols= ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
train[Gar_str_cols]=train[Gar_str_cols].fillna('None')

train['GarageYrBlt']=train['GarageYrBlt'].fillna(0)

In [ ]:
nan_percent= missing_percent(train)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
nan_percent.index

In [ ]:
train[['LotFrontage', 'FireplaceQu', 'Fence', 'Alley', 'MiscFeature',
       'PoolQC']]

In [ ]:
train= train.drop(['Fence', 'Alley', 'MiscFeature','PoolQC'], axis=1)

In [ ]:
nan_percent= missing_percent(train)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
#Filling in Fireplace Quality based on dataset documentation:
train['FireplaceQu']= train['FireplaceQu'].fillna('None')

In [ ]:
nan_percent= missing_percent(train)

plt.figure(figsize=(12,6))
sns.barplot(x=nan_percent.index, y=nan_percent)
plt.xticks(rotation=90)

In [ ]:
train['Neighborhood'].unique()

In [ ]:
plt.figure(figsize=(8,12))
sns.boxplot(data=train, x='LotFrontage', y='Neighborhood')

In [ ]:
#Impute missing data based on other columns:

train.groupby('Neighborhood')['LotFrontage']

In [ ]:
train.groupby('Neighborhood')['LotFrontage'].mean()

In [ ]:
train.groupby('Neighborhood')['LotFrontage'].transform(lambda val: val.fillna(val.mean()))

In [ ]:
train['LotFrontage']=train.groupby('Neighborhood')['LotFrontage'].transform(lambda val: val.fillna(val.mean()))

In [ ]:
train['LotFrontage']= train['LotFrontage'].fillna(0)

In [ ]:
nan_percent= missing_percent(train)

In [ ]:
nan_percent

# Dealing with Categorical Data

In [ ]:
train['MSSubClass']

In [ ]:
train.info()

In [ ]:
train['MSSubClass'].unique()

In [ ]:
#Convert to String:
train['MSSubClass']= train['MSSubClass'].apply(str)

In [ ]:
train.info()
#or: df['MS SubClass'].dtype

# Creating Dummy Variables


In [ ]:
train.select_dtypes(include='object')

In [ ]:
df_num= train.select_dtypes(exclude='object')
df_obj= train.select_dtypes(include='object')

In [ ]:
df_num.info()

In [ ]:
df_obj.info()

In [ ]:
# Converting:
df_obj= pd.get_dummies(df_obj, drop_first=True)

In [ ]:
df_obj.shape

In [ ]:
Final_df= pd.concat([df_num, df_obj], axis=1)

In [ ]:
Final_df.head()

In [ ]:
Final_df.isnull()

In [ ]:
X=Final_df.drop(['SalePrice'],axis=1)
y=Final_df['SalePrice']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Catboost model, training, test

In [ ]:
from catboost import CatBoostRegressor as cbr

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model=cbr()

In [ ]:
#create the grid
grid = {'max_depth': [3,4,5],'n_estimators':[100,200,300]}

#Instantiate GridSearchCV
gscv = GridSearchCV (estimator = model, param_grid = grid,  cv = 5)

In [ ]:
#fit the model
gscv.fit(X_train,y_train)

In [ ]:
#returns the best score
print(gscv.best_score_)

In [ ]:
#returns the estimator with the best performance
print(gscv.best_estimator_)

In [ ]:
#returns the best parameters
print(gscv.best_params_)